In [1]:
from osgeo import gdal
from osgeo import ogr, osr
from tqdm import tqdm
import random
import time
import numpy
import requests
import os
import math
headers = {
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6,zh-TW;q=0.5',
'Cache-Control': 'max-age=0',
'Connection': 'keep-alive',
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36 Edg/101.0.1210.53'
}

In [2]:

def deg2num(lat_deg, lon_deg, zoom):
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int(math.ceil((lon_deg + 180.0) / 360.0 * n))
    ytile = int(math.ceil((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n))
    return (xtile, ytile)

def num2deg(xtile, ytile, zoom):
    n = 2.0 ** zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
    lat_deg = math.degrees(lat_rad)
    return (lat_deg, lon_deg)

mytk = '--------------------'
def downloadtile(row,col,zoom,outputFolder):
    row = str(row)
    col = str(col)
    zoom = str(zoom)
    server = random.randrange(0,7)
    pic_url = 'https://t'+str(server)+'.tianditu.gov.cn/img_w/wmts?SERVICE=WMTS&REQUEST=GetTile&VERSION=1.0.0&LAYER=img&STYLE=default&TILEMATRIXSET=w&FORMAT=tiles&TILECOL='+col+'&TILEROW='+row+'&TILEMATRIX='+zoom+'&tk='+mytk
    outName = str(col) + ".png"
    outPath = os.path.join(outputFolder,outName)
    if not os.path.exists(outPath):
        with open(outPath, 'wb') as handle:
            response = requests.get(pic_url,headers=headers)
            if not response.ok:
                print(response,row,col,zoom,pic_url)
            for block in response.iter_content(1024):
                if not block:
                    break
                handle.write(block)
            
def batchRun(ul,br,zoom,outFolder):
    """_summary_

    Args:
        ul (float[]): 左上角角点坐标（纬度，经度）[22.76, 113.65]
        br (float[]): 右下角角点坐标（纬度，经度）[22.75, 113.69]
        zoom (int): 级别 16 
        outFolder (string): 输出文件夹 r'F:\tiles'
    """
    ColMin,RowMin = deg2num(ul[0], ul[1],zoom)
    ColMax,RowMax = deg2num(br[0], br[1],zoom)
    totalTiles = (ColMax - ColMin + 1)*(RowMax - RowMin + 1)
    count = 0
    with tqdm(total=totalTiles, desc='downloading', leave=False,bar_format='{l_bar}{bar:50}{r_bar}{bar:-20b}') as pbar:
        for row in range(RowMin,RowMax+1):
            imgOutputFolder = outFolder
            if not os.path.exists(os.path.join(outFolder,str(row))):
                os.mkdir(os.path.join(outFolder,str(row)))
            imgOutputFolder = os.path.join(outFolder,str(row))
            for col in range(ColMin,ColMax+1):
                downloadtile(row,col,zoom,imgOutputFolder)
                pbar.update(1)

#分辨率计算：(0.025399998 * ScaleDenominator) / (96*111000) = 分辨率
#其中 0.025399998 为英寸与厘米的换算，ScaleDenominator参考WMTS的定义文件，96是dpi，111000是一度大概等于多少米
#一个像素等于多少米是0.0254/96
#以下六行是tfw的定义
# 0.0000053798 pixel size in the x-direction in map units (GSD)
# 0.0000000000 rotation about y-axis
# 0.0000000000 rotation about x-axis
# -0.0000053798 pixel size in the y-direction in map in map units (GSD)
# 113.6508178710 x-coordinate of the upper left corner of the image
# 22.7597198149 y-coordinate of the upper left corner of the image
# 16级别：2.3886569458 （米）
# 17级别：1.1943284729 （米）
# 18级别：0.5971642365 （米）
def mergeTiles(ul,br,zoom,tilesPath,newImgPath,pixelSize):
    """_summary_

    Args:
        ul (float[]): 左上角角点坐标（纬度，经度）[22.76, 113.65]
        br (float[]): 右下角角点坐标（纬度，经度）[22.75, 113.69]
        zoom (int): 级别 16
        tilesPath (string): 存储瓦片的文件夹，格式为 Row-Col.jpg r'F:\tiles'
        newImgPath (string): 输出合并图形的位置 r'F:\tiles\main1.tif'
    """
    tileSize = 256
    ColMin,RowMin = deg2num(ul[0], ul[1],zoom)
    ColMax,RowMax = deg2num(br[0], br[1],zoom)
    imgWidth = (ColMax - ColMin + 1)*tileSize
    imgHeight = (RowMax - RowMin + 1)*tileSize
    resultImage = numpy.zeros((imgHeight,imgWidth,3),dtype=int)
    cursor = [0,0]
    #------------
    dirs = os.listdir(tilesPath)
    newDirs = []
    for value in dirs:
        if os.path.isdir(os.path.join(tilesPath,value)):
            newDirs.append(int(value))
    newDirs.sort()
    #-------------
    for idx,folder in enumerate(newDirs):
        folder = str(folder)
        cursor[0] = idx * tileSize
        cursor[1] = 0
        if os.path.isdir(os.path.join(tilesPath,folder)):
            #-----------------
            imgFiles = os.listdir(os.path.join(tilesPath,folder))
            newImageFiles = []
            for value in imgFiles:
                newImageFiles.append(int(value.split(".")[0]))
            newImageFiles.sort()
            #------------------
            for imgFile in newImageFiles:
                try:
                    imgFile = gdal.Open(os.path.join(tilesPath,folder,str(imgFile) + ".png"))
                    imgFile = [imgFile.GetRasterBand(1).ReadAsArray(),imgFile.GetRasterBand(2).ReadAsArray(),imgFile.GetRasterBand(3).ReadAsArray()]
                    resultImage[cursor[0]:cursor[0]+tileSize,cursor[1]:cursor[1]+ tileSize,:] = numpy.transpose(numpy.array(imgFile),(1,2,0))
                except Exception as ex:
                    print(ex)
                cursor[1] += tileSize
    driver = gdal.GetDriverByName('GTiff')
    ds = driver.Create(newImgPath, imgWidth, imgHeight, 3, gdal.GDT_Byte, options=['COMPRESS=JPEG', 'TILED=YES','PHOTOMETRIC=YCBCR'])
    inSpatialRef = osr.SpatialReference()
    inSpatialRef.ImportFromEPSG(4490)
    ds.SetProjection(inSpatialRef.ExportToWkt())
    y_max,x_min = num2deg(ColMin,RowMin,zoom)
    pixel_size = pixelSize
    ds.SetGeoTransform((x_min, pixel_size, 0, y_max, 0, -pixel_size))
    for b in range(3):
        ds.GetRasterBand(b+1).WriteArray(resultImage[:,:,b])
    ds.FlushCache()
    

In [6]:
dcount = 0
for root,dirs,files in os.walk('/home/pci/xuweijie/tiles'):
    for f in files:
        fullpath = os.path.join(root,f)
        if os.path.getsize(fullpath) < 200:
            os.remove(fullpath)
            dcount += 1
print("delete " + str(dcount))

delete 1


In [5]:
#xmin,xmax,ymin,ymax
gba = [["440704","江海区",113.06430085000432,113.18165646217608,22.485959394010877,22.607288374658822],
["440303","罗湖区",114.0611518628682,114.2151626680228,22.52962391021532,22.619249523346866],
["440783","开平市",112.23038487306349,112.81253514677962,21.972785309994094,22.668920078199108],
["440703","蓬江区",112.91640040395555,113.16522515231895,22.552047715829474,22.81052429236223],
["440784","鹤山市",112.47144019480925,113.04403886080479,22.469367787193676,22.85929604677517],
["440606","顺德区",113.00381705641212,113.38789308299943,22.676127246875012,23.010866198016686],
["440608","高明区",112.38470660982227,112.91843774880317,22.6452507825976,23.016298222784826],
["440604","禅城区",112.97448989019631,113.16305549291667,22.924806934592425,23.066262442257937],
["440113","番禺区",113.2440260000999,113.56991195307421,22.858774898642707,23.082744064686892],
["440105","海珠区",113.23674357435652,113.41457108838506,23.044832475638316,23.116404862394518],
["440103","荔湾区",113.17172337373472,113.2718661517016,23.044322869404937,23.162041509108406],
["440104","越秀区",113.23512543139361,113.31759339072244,23.10795524703707,23.17199321621399],
["441202","端州区",112.39372381700059,112.54314092021741,23.039840168089313,23.175028336388664],
["440106","天河区",113.29381735818458,113.44242877856539,23.100593797865542,23.248337583812486],
["441203","鼎湖区",112.4425210384153,112.77518254573795,23.092762664265592,23.316374095288324],
["440605","南海区",112.83293732409834,113.25765776539367,22.80087148483119,23.316788993916987],
["440112","黄埔区",113.39253121594288,113.60404172744381,23.03411670092504,23.415503441630563],
["441302","惠城区",114.1189243120075,114.78860372807969,22.91723645869564,23.416337614089684],
["440111","白云区",113.14284872209089,113.50509590215427,23.14072247296383,23.431152942054666],
["441204","高要区",112.18003861798786,112.83148263741384,22.771797508348357,23.437028540334893],
["441226","德庆县",111.5283398862768,112.26652022372946,23.074400063911867,23.511496896964275],
["440607","三水区",112.75926655730586,113.05313409586188,22.95876246711424,23.57598171768217],
["440114","花都区",112.95299784697318,113.47048652473336,23.248711039882608,23.61690192388471],
["440118","增城区",113.5417108864483,113.99619499790094,23.08679435757358,23.62237546225782],
["441284","四会市",112.42313677134496,112.87841974787318,23.20294511742327,23.694836276672902],
["441322","博罗县",113.83919041440618,114.69486547296322,23.050014559024703,23.709393435358777],
["440117","从化区",113.27462111597174,114.05542111569196,23.370180472151105,23.935699333257674],
["441324","龙门县",113.80773097990084,114.41682356525541,23.334761274113532,23.963082336081015],
["441223","广宁县",112.05780668383147,112.73782976033522,23.36481995697551,23.98153946957092],
["441225","封开县",111.35651131434577,112.02855541320184,23.2141804421737,23.9820779341494],
["441224","怀集县",111.86533239727237,112.51414956714882,23.45881491637533,24.391602019117954],
["440311","光明区",113.85306569902286,114.00421846141296,22.68885381395239,22.835763342903988],
["440309","龙华区",113.96168993913943,114.10830615330644,22.583652763164537,22.77452279678971],
["440310","坪山区",114.26243542255679,114.44060687870456,22.621338726700913,22.784201591130568],
["440115","南沙区",113.29089396900007,113.69951879499999,22.55247291700008,22.911562439000026],
["440304","福田区",113.98461992600005,114.10005368400004,22.500336080000068,22.590250127000047],
["440305","南山区",113.7817777030001,114.02147066100007,22.393599967000092,22.655445753000063],
["440306","宝安区",113.73421753100003,113.97543916600009,22.522093064000074,22.864709807000054],
["440307","龙岗区",114.04500317800009,114.623723913,22.44847268700009,22.816491883000005],
["440308","盐田区",114.19778966700008,114.34561834500002,22.543635328000093,22.656682990000025],
["440402","香洲区",113.40852302499998,114.31904777800007,21.80908354900008,22.538485400000045],
["440403","斗门区",113.07953229500006,113.414458338,22.082743967000066,22.407364771000086],
["440404","金湾区",113.10123493100002,113.48869682500006,21.835570700000055,22.189033334000012],
["440705","新会区",112.79040284700011,113.25754363800002,22.093124421000027,22.58012434400007],
["440781","台山市",112.29838644200007,113.047489045,21.566832016000063,22.444773374000075],
["440785","恩平市",111.99793845100004,112.524097299,21.915585677000024,22.495208088000027],
["441303","惠阳区",114.22409078800003,114.7038851770001,22.403188211000018,23.138104022000043],
["441323","惠东县",114.55396204500006,115.42164316500006,22.50528490500007,23.38825702200004],
["442000","中山市",113.15094363500009,113.62869741400004,22.20581074100005,22.776212706000024]]

# for item in gba:
#     outputTilesPath = os.path.join("/home/pci/xuweijie/tiles",item[0])
#     if not os.path.exists(outputTilesPath):
#         os.mkdir(outputTilesPath)
#     batchRun([item[5], item[2]],[item[4], item[3]],12,outputTileWsPath)
#     mergeTiles([item[5], item[2]],[item[4], item[3]],12,outputTilesPath,r"/home/pci/xuweijie/tiles/%s.tif"%item[1])
# 16级别：2.3886569458 （米）
# 17级别：1.1943284729 （米）
# 18级别：0.5971642365 （米）
item = ["440111","白云区",113.14284872209089,113.50509590215427,23.14072247296383,23.431152942054666]
scale2resolution = {
    '16':0.00002152,
    '17':0.00001076,
    '18':0.00000538
}
for value in [18]:
    outputTilesPath = os.path.join("/home/pci/xuweijie/tiles",item[0] + str(value))
    if not os.path.exists(outputTilesPath):
        os.mkdir(outputTilesPath)
    batchRun([item[5], item[2]],[item[4], item[3]],value,outputTilesPath)
    mergeTiles([item[5], item[2]],[item[4], item[3]],value,outputTilesPath,r"/home/pci/xuweijie/tiles/%s_%d.tif"%(item[1],value),scale2resolution[str(value)])

In [134]:
num2deg(213830,114045,18)

(22.759719814922637, 113.65081787109375)

In [10]:
#天地图key：8bdb7ae7c4db0f28379fdf0279a50ae6
pic_url = 'https://t0.tianditu.gov.cn/img_w/wmts?SERVICE=WMTS&REQUEST=GetTile&VERSION=1.0.0&LAYER=img&STYLE=default&TILEMATRIXSET=w&FORMAT=tiles&TILECOL=53444&TILEROW=28445&TILEMATRIX=16&tk=8bdb7ae7c4db0f28379fdf0279a50ae6'
with open('F:/pic1.jpg', 'wb') as handle:
    response = requests.get(pic_url,headers=headers)
    if not response.ok:
        print(response)
    for block in response.iter_content(1024):
        if not block:
            break
        handle.write(block)